In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
! pip install pmdarima # this package would be needed for AutoARIMA

In [ ]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import pmdarima as pm

import seaborn as sns

%matplotlib inline
plt.style.use('ggplot')

# Error

In [ ]:
def errors(prediction,actual):
    
    mae = np.mean(np.abs(prediction - actual))
    
    mape = np.mean(np.abs(prediction - actual)/np.abs(actual))
    
    rmse = np.mean((prediction - actual)**2)**0.5
    
    return({'mae':mae,'mape':mape,'rmse':rmse})

# ADF Test

In [ ]:
def adf_test(series):
    result = adfuller(series.dropna())
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    return result

In [ ]:
df = pd.read_csv("../input/international-airline-passengers/international-airline-passengers.csv")

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.drop(df.index[144],inplace=True)

In [ ]:
df.columns=['Month','No of Passangers']

In [ ]:
df.tail()

In [ ]:
df['Month']=pd.to_datetime(df['Month'],format='%Y-%m')

In [ ]:
type(df['Month'][0])

In [ ]:
df.index=df['Month']

In [ ]:
df.head()

In [ ]:
type(df)

In [ ]:
df.drop('Month',axis=1,inplace=True)

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
 rolling_mean = df['No of Passangers'].rolling(window=12).mean()
 

In [ ]:
plt.plot(df['No of Passangers'],label='Actual Count')
plt.plot(rolling_mean, color='blue', linestyle='dashed',label='Rolling Mean')
plt.legend(['Actual Count', 'Rolling Mean'],loc='upper left')
plt.xlabel('Month')
plt.ylabel('No of Passangers')
plt.show()

## The series has an upward trend and an yearly seasonality

[Seasonal Decompose Documentation](https://www.statsmodels.org/stable/generated/statsmodels.tsa.seasonal.seasonal_decompose.html)

In [ ]:
components = seasonal_decompose(df, model='additive')

In [ ]:
components.plot()
plt.show()

In [ ]:
#https://www.statsmodels.org/stable/generated/statsmodels.tsa.seasonal.seasonal_decompose.html

### Observation:the series has a trend and seasonality and differencing might be needed to make it stationary. Let's first check its stationarity. Also, let's consider a significance=0.05

In [ ]:
series=df['No of Passangers'] # Creating the Series

In [ ]:
result=adf_test(series)

In [ ]:
#result

### p>alpha(significance); We can't reject the null hypothesis.We accept the null hypothesis that the series is not stationary. Now, we need to make it stationary

In [ ]:
# Line and ACF Plot of the Original Series
fig, axes = plt.subplots(2)
axes[0].plot(series); axes[0].set_title('Original Series')
plot_acf(series, ax=axes[1])
plt.show()

In [ ]:
# Line, ACF, and PCF Plot of the Original Series after first USUAL differencing
plt.rcParams.update({'figure.figsize':(9,3), 'figure.dpi':120})
fig, axes = plt.subplots(3)
axes[0].plot(series.diff()); axes[0].set_title('1st Usual Differencing')
#axes[1].set(ylim=(0,2))
plot_acf(series.diff().dropna(), ax=axes[1])
plot_pacf(series.diff().dropna(), ax=axes[2])

plt.show()

In [ ]:
# Performing the ADF-test of the series after first differencing
result2=adf_test(series.diff(12))

In [ ]:
# p<0.05; d=1

In [ ]:
# Line, ACF, and PCF Plot of the Original Series after first SEASONAL differencing
fig, axes = plt.subplots(3)
axes[0].plot(series.diff(12)); axes[0].set_title('1st Seasonal Differencing')
plot_acf(series.diff(12).dropna(), ax=axes[1])
plot_pacf(series.diff(12).dropna(), ax=axes[2])

plt.show()

# p=1, d=1,q=1; P=1,D=1,Q=3, s=12
## Final Model:
# SARIMA(1,1,1)X(1,1,3,12)

# Seasonal ARIMA

In [ ]:
# Splitting the series into train and test(last 24 months of test data)

In [ ]:
train=series[:-24]
test=series[-24:]

In [ ]:
import statsmodels.api as sm

In [ ]:
mod = sm.tsa.statespace.SARIMAX(series, order=(1,1,1), seasonal_order=(1,1,3,12))

In [ ]:
res=mod.fit()

In [ ]:
# validating on the last 24 months of test data

In [ ]:
forecast=res.predict(start=120,end=143,dynamic=True)

In [ ]:
#Error in manual calculation 
errors_manual=errors(forecast,test) #test=series[120:]
errors_manual

In [ ]:
plt.plot(series,label = 'Actual')
plt.plot(forecast,label = 'Predicted')
plt.legend(['Actual','Predicted'], loc='upper left')
plt.show()

# Auto ARIMA

[Auto Arima Documentation](http://alkaline-ml.com/pmdarima/1.6.1/modules/generated/pmdarima.arima.auto_arima.html)

In [ ]:
model = pm.auto_arima(train, start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=12,              # periodicity of 12 months as the data timeline is in months
                      d=None,           # let the model determine 'd'
                      seasonal=True,   # Seasonality
                      start_P=0, 
                      D=1, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model.summary())

In [ ]:
model.aic()

In [ ]:
model.fit(train)

In [ ]:
model.aic()

In [ ]:
val_forecast=model.predict(n_periods=len(test))

In [ ]:
val_forecast

### Note:
In case of Auto Arima the forecast has to be supplied as n_periods BUT for normal ARIMA is has to be supplied as (start,end,dynamic=True)

In [ ]:
val_forecast=pd.Series(val_forecast,index=test.index)
#val_forecast

In [ ]:
errors_auto=errors(val_forecast,test)
errors_auto

In [ ]:
plt.plot(val_forecast,label = 'Predicted')
plt.plot(series,label = 'Train')
plt.plot(test,label = 'Test')
plt.legend(['Predicted','Train','Test'], loc='upper left')
plt.show()

# Forecast

Train on the whole dataset and forecast for the immediate next 24 months

In [ ]:
model = pm.auto_arima(series, start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=12,              # periodicity=12
                      d=None,           # let model determine 'd'
                      seasonal=True,   # Seasonality
                      start_P=0, 
                      D=1, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model.summary())

In [ ]:
from pandas.tseries.offsets import DateOffset
n_periods=24 # Forecasting next 24 months
# fc, confint = model.predict(n_periods=n_periods, return_conf_int=True) # use for plotting confidence band also
fc= model.predict(n_periods=n_periods)
#Creating index for the forecast period
fc_index=[series.index[-1]+DateOffset(months=x) for x in range(1,n_periods+1)]
fc_series = pd.Series(fc, index=fc_index)

In [ ]:
len(fc_index)

In [ ]:
#fc_index

In [ ]:
#confint

In [ ]:
plt.plot(fc_series,label = 'forecast')
plt.plot(series,label = 'total')
plt.legend(['forecast','total'], loc='upper left')
plt.show()